# Sales Database Demo - Render Connection

This notebook demonstrates connecting to the Render PostgreSQL database and running queries.


In [ ]:
# Import required libraries
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import os

print("✅ Libraries imported successfully")


In [ ]:
# Load environment variables and connect to database
load_dotenv()

def get_db_url():
    POSTGRES_SERVER = os.getenv("POSTGRES_SERVER", "")
    
    if POSTGRES_SERVER.startswith("postgresql://") or POSTGRES_SERVER.startswith("postgres://"):
        return POSTGRES_SERVER
    else:
        POSTGRES_USERNAME = os.getenv("POSTGRES_USERNAME")
        POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
        POSTGRES_DATABASE = os.getenv("POSTGRES_DATABASE")
        return f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_SERVER}/{POSTGRES_DATABASE}"

DATABASE_URL = get_db_url()
conn = psycopg2.connect(DATABASE_URL)

print("✅ Connected to Render database successfully!")


## Query 1: Show All Tables and Row Counts


In [ ]:
query = """
SELECT 
    'region' as table_name, COUNT(*) as row_count FROM region
UNION ALL SELECT 'country', COUNT(*) FROM country
UNION ALL SELECT 'productcategory', COUNT(*) FROM productcategory
UNION ALL SELECT 'customer', COUNT(*) FROM customer
UNION ALL SELECT 'product', COUNT(*) FROM product
UNION ALL SELECT 'orderdetail', COUNT(*) FROM orderdetail
ORDER BY row_count DESC;
"""

df = pd.read_sql_query(query, conn)
print("📋 Tables and Row Counts:")
df


## Query 2: Total Sales by Region


In [ ]:
query = """
SELECT
    r.region AS Region,
    COUNT(DISTINCT c.customerid) AS TotalCustomers,
    COUNT(od.orderid) AS TotalOrders,
    ROUND(CAST(SUM(p.productunitprice * od.quantityordered) AS NUMERIC), 2) AS TotalRevenue
FROM orderdetail od
JOIN customer c ON od.customerid = c.customerid
JOIN product p ON od.productid = p.productid
JOIN country co ON c.countryid = co.countryid
JOIN region r ON co.regionid = r.regionid
GROUP BY r.regionid, r.region
ORDER BY TotalRevenue DESC;
"""

df = pd.read_sql_query(query, conn)
print("💰 Total Sales by Region:")
df


## Query 3: Top 10 Products by Revenue


In [ ]:
query = """
SELECT
    p.productname AS ProductName,
    pc.productcategory AS Category,
    COUNT(od.orderid) AS TimesSold,
    ROUND(CAST(SUM(p.productunitprice * od.quantityordered) AS NUMERIC), 2) AS TotalRevenue
FROM orderdetail od
JOIN product p ON od.productid = p.productid
JOIN productcategory pc ON p.productcategoryid = pc.productcategoryid
GROUP BY p.productid, p.productname, pc.productcategory
ORDER BY TotalRevenue DESC
LIMIT 10;
"""

df = pd.read_sql_query(query, conn)
print("📦 Top 10 Products:")
df


In [ ]:
conn.close()
print("✅ Connection closed")
